In [1]:
import cv2
import numpy as np
from sklearn.svm import SVC
import os
import sys
import time
from skimage.feature import hog
from skimage import data, exposure

In [4]:
clases = os.listdir("raw-img")
print("CLASES:" , len(clases) , clases)

CLASES: 10 ['perro', 'caballo', 'mariposa', 'vaca', 'gallina', 'ardilla', 'elefante', 'araña', 'gato', 'oveja']


In [ ]:
datasets = ["raw-img"]

HEIGHT =  200
WIDTH = 300

N = 10

num_blocks_rows = HEIGHT // N
num_blocks_cols = WIDTH // N

for dataset in datasets:
    print("dataset: ", dataset)
    # Iniciamos el contador de tiempo
    startTimer = time.perf_counter()

    # Definimos el directorio de entrenamiento y prueba
    train_dir = dataset+"/"+"train"
    test_dir = dataset+"/"+"test"

    # Inicializamos los arrays de datos de entrenamiento y prueba
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    print("Procesamos las imágenes de entrenamiento")
    # Procesamos las imágenes de entrenamiento
    for label in os.listdir(train_dir):
        label_dir = os.path.join(train_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path , 0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)            
            
            
            features = feature_extraction(resized_image, N);       

            X_train.append(features.flatten())
            y_train.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtrain ... (" + str(int(progress)) + " %)")
        print("")

    print("Procesamos las imágenes de prueba")
    # Procesamos las imágenes de prueba
    for label in os.listdir(test_dir):
        label_dir = os.path.join(test_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path,0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
                
            features = feature_extraction(resized_image, N);   

            
            X_test.append(features.flatten())
            y_test.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtest ... (" + str(int(progress)) + " %)")
        print("")
    print("")
    
    # Convertimos los datos de entrenamiento y prueba a arrays de NumPy
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)


    print("SHAPE Xtrin",X_train.shape)
    print("SHAPE y_train",y_train.shape)
    print("SHAPE X_test",X_test.shape)
    print("SHAPE y_test",y_test.shape)
    
    
    # Creamos un clasificador SVM
    clf = SVC(gamma='auto')

    # Entrenamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, y_train)

    # Realizamos predicciones con el clasificador entrenado
    y_pred = clf.predict(X_test)

    # Calculamos el porcentaje de acierto
    accuracy = np.mean(y_pred == y_test)
    print("Precisión: {:.2f}%".format(accuracy * 100))

    # Detenemos el contador de tiempo
    endTimer = time.perf_counter()

    # Calculamos el tiempo transcurrido
    elapsed = endTimer - startTimer
    # Obtenemos el tiempo transcurrido en minutos y segundos
    minutes, seconds = divmod(elapsed, 60)

    # Mostramos el tiempo transcurrido en minutos y segundos
    print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.2f} segundos")